In [1]:
import requests
from bs4 import BeautifulSoup as BS


In [111]:
def prettify_page(url):
    headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}
    page = requests.get(url, headers=headers, timeout=5)
    soup = BS(page.content, 'html.parser').html
    return soup

In [112]:
page_1 = prettify_page('https://www.imdb.com/search/title/?title_type=feature&release_date=2017-01-01,2019-12-31&countries=us&sort=boxoffice_gross_us,desc&count=250&ref_=adv_prv')

In [113]:
page_2 = prettify_page('https://www.imdb.com/search/title/?title_type=feature&release_date=2017-01-01,2019-12-31&countries=us&sort=boxoffice_gross_us,desc&count=250&start=251&ref_=adv_nxt')

In [132]:
def parse_movies(soup):
    movies_dict = {}
    items = soup.find_all('div', class_='lister-item')
    
    for item in items:
        
        movie_id = item.find('h3', class_='lister-item-header')
        movie_id = movie_id.find('a')
        movie_id = movie_id['href']
        
        movie_id = movie_id.replace('/title/', '')
        movie_id = movie_id.replace('/', '')
        movie_id = movie_id.replace('?ref_=adv_li_tt', "")
        
        title = item.find('h3', class_='lister-item-header')
        title = title.find('a').string
        
        
        user_rating = item.find('div', class_='ratings-imdb-rating') 
        if user_rating:
            user_rating = user_rating.find('strong').string
        else:
            user_rating = -1
            
        user_rating = int(float(user_rating) * 10) 
        
        metascore = item.find('div', class_ = 'ratings-metascore')
        if metascore: 
            metascore = metascore.find('span', class_ = "metascore").string.strip()
        else:
            metascore = -1
            
        metascore = int(metascore)     
        
        movies_dict[movie_id] = (title, user_rating, metascore)
        
    return movies_dict
        


In [133]:
new_dict_1 = parse_movies(page_1)
new_dict_1

{'tt4154796': ('Avengers: Endgame', 85, 78),
 'tt1825683': ('Black Panther', 73, 88),
 'tt4154756': ('Avengers: Infinity War', 85, 68),
 'tt2527336': ('Star Wars: Episode VIII - The Last Jedi', 71, 85),
 'tt3606756': ('Incredibles 2', 77, 80),
 'tt6105098': ('The Lion King', 70, 55),
 'tt2771200': ('Beauty and the Beast', 72, 65),
 'tt1979376': ('Toy Story 4', 79, 84),
 'tt4154664': ('Captain Marvel', 69, 64),
 'tt4881806': ('Jurassic World: Fallen Kingdom', 62, 51),
 'tt0451279': ('Wonder Woman', 74, 76),
 'tt2283362': ('Jumanji: Welcome to the Jungle', 69, 58),
 'tt3896198': ('Guardians of the Galaxy Vol. 2', 76, 67),
 'tt6320628': ('Spider-Man: Far from Home', 76, 69),
 'tt6139732': ('Aladdin', 71, 53),
 'tt1477834': ('Aquaman', 70, 55),
 'tt2250912': ('Spider-Man: Homecoming', 74, 73),
 'tt1396484': ('It', 73, 69),
 'tt5463162': ('Deadpool 2', 77, 66),
 'tt3501632': ('Thor: Ragnarok', 79, 74),
 'tt2709692': ("Dr. Seuss' The Grinch", 63, 51),
 'tt3469046': ('Despicable Me 3', 63, 49

In [186]:
new_dict_2 = parse_movies(page_2)
new_dict_2

{'tt7734218': ('Stuber', 62, 42),
 'tt5816682': ('Victoria & Abdul', 68, 58),
 'tt4799050': ('Rough Night', 52, 51),
 'tt2274648': ('Hellboy', 52, 31),
 'tt4131800': ('My Little Pony: The Movie', 61, 39),
 'tt4530422': ('Overlord', 66, 60),
 'tt4463894': ('Shaft', 64, 40),
 'tt3521126': ('The Disaster Artist', 74, 76),
 'tt7329656': ('47 Meters Down: Uncaged', 50, 43),
 'tt5776858': ('Phantom Thread', 75, 90),
 'tt6421110': ('Proud Mary', 49, 35),
 'tt2072233': ('Sleepless', 56, 34),
 'tt1308728': ('The Happytime Murders', 54, 27),
 'tt6003368': ('Diary of a Wimpy Kid: The Long Haul', 43, 39),
 'tt6324278': ('Abominable', 70, 61),
 'tt1025100': ('Gemini Man', 57, 38),
 'tt7690670': ('SuperFly', 51, 52),
 'tt4287320': ('The Circle', 53, 43),
 'tt1946502': ('UglyDolls', 48, 39),
 'tt9024106': ('Unplanned', 58, 10),
 'tt1206885': ('Rambo: Last Blood', 64, 26),
 'tt0493405': ('CHIPS', 60, 28),
 'tt3829920': ('Only The Brave', 77, 72),
 'tt6628394': ('Bad Times at the El Royale', 71, 60),
 

In [181]:
import mysql.connector
import config
cnx = mysql.connector .connect(
    host = config.host,
    user = config.user,
    passwd = config.pw,
    database = 'movie_db'
)

cursor = cnx.cursor()


In [57]:
def close_db():
    cursor.close()
    cnx.close()

In [123]:
def add_to_table(movie_dict):
    for movie_id in movie_dict.keys():
        data_movie = {
      'movie_id': movie_id,
      'title': movie_dict[movie_id][0],
      'user_rating': movie_dict[movie_id][1],
      'metascore': movie_dict[movie_id][2],
        }
        #print(data_movie)
        
        inserting = """INSERT INTO Master_Table  
         (movie_id, title, user_rating, metascore) 
         VALUES (%(movie_id)s, %(title)s, %(user_rating)s, %(metascore)s);"""
        
        cursor.execute(inserting, data_movie)
        cnx.commit()
        
        

In [64]:
add_to_table(new_dict_2)

In [ ]:
#<span class=“ipl-status-pill ipl-status-pill--ok”>Mild</span>

In [143]:
def parse_parent_guide(movie_id):
    
    categories = {}
    url = "https://www.imdb.com/title/" + movie_id + "/parentalguide?ref_=tt_stry_pg"
    soup = prettify_page(url)
    
    
    mpaa_rating = soup.find(id="mpaa-rating")
    
    if mpaa_rating:
        mpaa_rating = mpaa_rating.find_all("td")[1].string
    else:
        mpaa_rating = "None"
    
    
    nudity = soup.find('section', id='advisory-nudity')
    
    if nudity:
        nudity = nudity.find('span', class_='ipl-status-pill').string
    else:
        nudity = 'No Data'
    
    
    violence = soup.find('section', id='advisory-violence')
    
    if violence:
        violence = violence.find('span', class_='ipl-status-pill').string
    else:
        violence = 'No Data'
    
    profanity = soup.find('section', id='advisory-profanity')
    
    if profanity:
        profanity = profanity.find('span', class_='ipl-status-pill').string
    else:
        profanity = "No Data"
    
    alcohol = soup.find('section', id='advisory-alcohol')
    
    if alcohol:
        alcohol = alcohol.find('span', class_='ipl-status-pill').string
    else:
        alcohol = "No Data"
    
    frightening = soup.find('section', id='advisory-frightening')
    
    if frightening:
        frightening = frightening.find('span', class_='ipl-status-pill').string
    else:
        fightening = "No Data"
        
    categories[movie_id] = (mpaa_rating, nudity, violence, profanity, alcohol, frightening)
    
    return categories
    
    #print(f"nudity: {nudity}, violence: {violence}, profanity: {profanity}, alcohol: {alcohol}, frightening: {frightening}")
  

In [144]:
parse_parent_guide('tt4154796')

{'tt4154796': ('Rated PG-13 for sequences of sci-fi violence and action, and some language',
  'None',
  'Moderate',
  'Mild',
  'Mild',
  'Moderate')}

In [128]:
import requests
import time

In [184]:
def parse_all_categories(movie_dict):
    counter = 0
    categories = {}
    for movie_id in movie_dict.keys():
        categories = parse_parent_guide(movie_id)[movie_id]
        
        
        inserting = f"UPDATE Master_Table SET MPAA_rating = '{categories[0]}', nudity = '{categories[1]}', violence = '{categories[2]}', profanity = '{categories[3]}', alcohol = '{categories[4]}', frightening = '{categories[5]}' WHERE movie_id = '{movie_id}'"
        

        cursor.execute(inserting)
        cnx.commit()

        print(inserting)
        
        if counter%20 == 0:
            time.sleep(10)
        else:
            time.sleep(1)
        counter += 1
        
    return categories

In [185]:
#parse_all_categories(new_dict_1)

UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for sequences of sci-fi violence and action, and some language', nudity = 'None', violence = 'Moderate', profanity = 'Mild', alcohol = 'Mild', frightening = 'Moderate' WHERE movie_id = 'tt4154796'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for prolonged sequences of action violence, and a brief rude gesture', nudity = 'None', violence = 'Moderate', profanity = 'Mild', alcohol = 'None', frightening = 'Moderate' WHERE movie_id = 'tt1825683'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for intense sequences of sci-fi violence and action throughout, language and some crude references.', nudity = 'None', violence = 'Moderate', profanity = 'Mild', alcohol = 'None', frightening = 'Moderate' WHERE movie_id = 'tt4154756'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for sequences of sci-fi action and violence.', nudity = 'None', violence = 'Moderate', profanity = 'Mild', alcohol = 'Mild', frightening = 'Moderate' WHERE movie_

UPDATE Master_Table SET MPAA_rating = 'Rated PG for frenetic sequences of animated action violence, thematic elements, and mild language', nudity = 'None', violence = 'Mild', profanity = 'Mild', alcohol = 'None', frightening = 'Mild' WHERE movie_id = 'tt4633694'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for intense war experience and some language', nudity = 'None', violence = 'Moderate', profanity = 'Moderate', alcohol = 'Mild', frightening = 'Severe' WHERE movie_id = 'tt5013056'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for terror and some bloody images', nudity = 'None', violence = 'Moderate', profanity = 'None', alcohol = 'None', frightening = 'Severe' WHERE movie_id = 'tt6644200'
UPDATE Master_Table SET MPAA_rating = 'Rated R for violence, bloody images, and language including sexual references', nudity = 'Mild', violence = 'Moderate', profanity = 'Severe', alcohol = 'Mild', frightening = 'Moderate' WHERE movie_id = 'tt5052448'
UPDATE Master_Table SET MPAA_rating

UPDATE Master_Table SET MPAA_rating = 'Rated R for strong erotic sexual content, some graphic nudity, and language', nudity = 'Severe', violence = 'Mild', profanity = 'Moderate', alcohol = 'Mild', frightening = 'Mild' WHERE movie_id = 'tt4465564'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for violence including some bloody images, thematic elements, and language', nudity = 'None', violence = 'Moderate', profanity = 'Mild', alcohol = 'Mild', frightening = 'Moderate' WHERE movie_id = 'tt6823368'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for sequences of monster action violence and destruction, and for some language', nudity = 'None', violence = 'Moderate', profanity = 'Moderate', alcohol = 'None', frightening = 'Moderate' WHERE movie_id = 'tt3741700'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for suggestive content and drug use', nudity = 'Mild', violence = 'None', profanity = 'Mild', alcohol = 'Mild', frightening = 'Mild' WHERE movie_id = 'tt1987680'
UPDATE Mast

UPDATE Master_Table SET MPAA_rating = 'Rated R for horror violence and terror', nudity = 'None', violence = 'Moderate', profanity = 'Mild', alcohol = 'None', frightening = 'Severe' WHERE movie_id = 'tt8350360'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for crude sexual content, language, and drug references throughout', nudity = 'Severe', violence = 'Mild', profanity = 'Severe', alcohol = 'Moderate', frightening = 'None' WHERE movie_id = 'tt7054636'
UPDATE Master_Table SET MPAA_rating = 'Rated R for crude sexual  content and language throughout, and some drug use', nudity = 'Moderate', violence = 'None', profanity = 'Severe', alcohol = 'Moderate', frightening = 'None' WHERE movie_id = 'tt6359956'
UPDATE Master_Table SET MPAA_rating = 'Rated R for strong disturbing violence throughout, pervasive language, some sexuality and drug use', nudity = 'Mild', violence = 'Severe', profanity = 'Severe', alcohol = 'Moderate', frightening = 'Moderate' WHERE movie_id = 'tt6133466'
UPDATE Mas

UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for sexual material, drug content and partying', nudity = 'Mild', violence = 'Mild', profanity = 'Mild', alcohol = 'Mild', frightening = 'None' WHERE movie_id = 'tt5619332'
UPDATE Master_Table SET MPAA_rating = 'Rated R for sequences of strong violence, language throughout, and some sexuality/nudity', nudity = 'Moderate', violence = 'Severe', profanity = 'Moderate', alcohol = 'Moderate', frightening = 'Moderate' WHERE movie_id = 'tt2406566'
UPDATE Master_Table SET MPAA_rating = 'Rated R for language throughout and some sexuality/nudity', nudity = 'Mild', violence = 'Mild', profanity = 'Severe', alcohol = 'Moderate', frightening = 'Mild' WHERE movie_id = 'tt3532216'
UPDATE Master_Table SET MPAA_rating = 'Rated R for strong bloody violence, language throughout, and crude sexual references', nudity = 'Mild', violence = 'Severe', profanity = 'Severe', alcohol = 'Mild', frightening = 'Moderate' WHERE movie_id = 'tt3829266'
UPDATE Master_Tab

UPDATE Master_Table SET MPAA_rating = 'Rated PG for thematic elements, some peril and language', nudity = 'None', violence = 'Mild', profanity = 'Mild', alcohol = 'None', frightening = 'Mild' WHERE movie_id = 'tt7616798'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for sci-fi violence and action, suggestive material and brief language', nudity = 'Mild', violence = 'Mild', profanity = 'Mild', alcohol = 'None', frightening = 'None' WHERE movie_id = 'tt2239822'
UPDATE Master_Table SET MPAA_rating = 'Rated PG for some thematic elements', nudity = 'None', violence = 'Mild', profanity = 'None', alcohol = 'None', frightening = 'Mild' WHERE movie_id = 'tt4587656'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for thematic elements, sexual references, language and teen partying', nudity = 'Mild', violence = 'None', profanity = 'Mild', alcohol = 'Mild', frightening = 'None' WHERE movie_id = 'tt5164432'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for violence and disturbing conte

UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for thematic elements and some violent images', nudity = 'None', violence = 'Moderate', profanity = 'Mild', alcohol = 'None', frightening = 'Moderate' WHERE movie_id = 'tt5104604'
UPDATE Master_Table SET MPAA_rating = 'Rated R for sexual content and language throughout, and for drug material.', nudity = 'Moderate', violence = 'Mild', profanity = 'Moderate', alcohol = 'Moderate', frightening = 'None' WHERE movie_id = 'tt8097306'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for injury images, peril, language, brief drug use, partial nudity and thematic elements', nudity = 'Mild', violence = 'Mild', profanity = 'Mild', alcohol = 'Mild', frightening = 'Moderate' WHERE movie_id = 'tt6306064'
UPDATE Master_Table SET MPAA_rating = 'Rated PG for thematic elements, some suggestive material, and language', nudity = 'Mild', violence = 'None', profanity = 'None', alcohol = 'Mild', frightening = 'Mild' WHERE movie_id = 'tt6398184'
UPDATE Mast

UPDATE Master_Table SET MPAA_rating = 'Rated R for language including sexual references, some graphic nudity and drug use.', nudity = 'Moderate', violence = 'None', profanity = 'Moderate', alcohol = 'Moderate', frightening = 'None' WHERE movie_id = 'tt1389072'
UPDATE Master_Table SET MPAA_rating = 'Rated R for horror violence and terror throughout, and for language', nudity = 'Mild', violence = 'Severe', profanity = 'Moderate', alcohol = 'Mild', frightening = 'Severe' WHERE movie_id = 'tt1285009'
UPDATE Master_Table SET MPAA_rating = 'Rated R for language throughout, drug content, violence, some sexual references, and brief nudity', nudity = 'Mild', violence = 'Moderate', profanity = 'Severe', alcohol = 'Moderate', frightening = 'Moderate' WHERE movie_id = 'tt4537896'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for crude and sexual material, language throughout, some violence and drug content', nudity = 'Mild', violence = 'Mild', profanity = 'Mild', alcohol = 'Mild', frightening

('Rated PG-13 for terror, horror violence, bloody images, sexual content, thematic elements, partial nudity, some language and teen drinking',
 'Mild',
 'Moderate',
 'Moderate',
 'None',
 'Moderate')

In [187]:
#parse_all_categories(new_dict_2)

UPDATE Master_Table SET MPAA_rating = 'Rated R for violence and language throughout, some sexual references and brief graphic nudity', nudity = 'Moderate', violence = 'Moderate', profanity = 'Severe', alcohol = 'Mild', frightening = 'Mild' WHERE movie_id = 'tt7734218'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for some thematic elements and language', nudity = 'None', violence = 'Mild', profanity = 'Mild', alcohol = 'None', frightening = 'Mild' WHERE movie_id = 'tt5816682'
UPDATE Master_Table SET MPAA_rating = 'R for crude sexual content, language throughout, drug use and brief bloody images', nudity = 'Moderate', violence = 'Moderate', profanity = 'Moderate', alcohol = 'Moderate', frightening = 'Mild' WHERE movie_id = 'tt4799050'
UPDATE Master_Table SET MPAA_rating = 'Rated R for strong bloody violence and gore throughout, and language', nudity = 'Mild', violence = 'Severe', profanity = 'Moderate', alcohol = 'Moderate', frightening = 'Severe' WHERE movie_id = 'tt2274648'
UPDAT

UPDATE Master_Table SET MPAA_rating = 'Rated PG for thematic material, brief language and some smoking.', nudity = 'None', violence = 'None', profanity = 'Mild', alcohol = 'Mild', frightening = 'None' WHERE movie_id = 'tt8637428'
UPDATE Master_Table SET MPAA_rating = 'Rated PG - 13 for violence and terror, sexual  content, language, thematic material, and some drug  references', nudity = 'Moderate', violence = 'Moderate', profanity = 'Moderate', alcohol = 'Mild', frightening = 'Moderate' WHERE movie_id = 'tt2039338'
UPDATE Master_Table SET MPAA_rating = 'Rated PG for fantasy action violence, scary images, thematic elements including some bullying, and language', nudity = 'None', violence = 'Mild', profanity = 'Mild', alcohol = 'None', frightening = 'Mild' WHERE movie_id = 'tt6811018'
UPDATE Master_Table SET MPAA_rating = 'Rated R for strong violence and pervasive language', nudity = 'Moderate', violence = 'Severe', profanity = 'Severe', alcohol = 'Mild', frightening = 'Severe' WHERE mo

UPDATE Master_Table SET MPAA_rating = 'Rated PG for some thematic elements', nudity = 'None', violence = 'None', profanity = 'None', alcohol = 'Mild', frightening = 'None' WHERE movie_id = 'tt5427194'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for thematic material, racial epithets, some violence and a suggestive reference', nudity = 'Mild', violence = 'None', profanity = 'Moderate', alcohol = 'Moderate', frightening = 'Mild' WHERE movie_id = 'tt4807408'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for mature thematic content, sexuality, violence and some strong language.', nudity = 'Mild', violence = 'Mild', profanity = 'Moderate', alcohol = 'Moderate', frightening = 'Mild' WHERE movie_id = 'tt5301662'
UPDATE Master_Table SET MPAA_rating = 'Rated R for disturbing violence throughout, bloody images, and language', nudity = 'Mild', violence = 'Severe', profanity = 'Moderate', alcohol = 'Mild', frightening = 'Severe' WHERE movie_id = 'tt5461944'
UPDATE Master_Table SET MPAA

UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for thematic content, language, teen drinking, and suggestive material', nudity = 'Mild', violence = 'None', profanity = 'Mild', alcohol = 'Mild', frightening = 'None' WHERE movie_id = 'tt7026672'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for language, suggestive material and brief violence', nudity = 'Moderate', violence = 'Moderate', profanity = 'Moderate', alcohol = 'Moderate', frightening = 'None' WHERE movie_id = 'tt5721088'
UPDATE Master_Table SET MPAA_rating = 'Rated PG for thematic material throughout', nudity = 'None', violence = 'Mild', profanity = 'None', alcohol = 'Mild', frightening = 'Mild' WHERE movie_id = 'tt3892172'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for sci-fi violence and action, some sexual content, brief language and drug material', nudity = 'Mild', violence = 'Moderate', profanity = 'Moderate', alcohol = 'Mild', frightening = 'Moderate' WHERE movie_id = 'tt5968394'
UPDATE Master_Table SET 

UPDATE Master_Table SET MPAA_rating = 'None', nudity = 'None', violence = 'None', profanity = 'None', alcohol = 'None', frightening = 'None' WHERE movie_id = 'tt7935892'
UPDATE Master_Table SET MPAA_rating = 'Rated R for pervasive drug and alcohol use, language throughout, nudity and some strong sexual content', nudity = 'Severe', violence = 'Mild', profanity = 'Severe', alcohol = 'Severe', frightening = 'Mild' WHERE movie_id = 'tt6511932'
UPDATE Master_Table SET MPAA_rating = 'Rated R for some strong sexuality', nudity = 'Severe', violence = 'None', profanity = 'Mild', alcohol = 'Mild', frightening = 'None' WHERE movie_id = 'tt6108178'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for thematic elements involving domestic abuse, violence, some sexuality and language', nudity = 'Mild', violence = 'Severe', profanity = 'None', alcohol = 'Mild', frightening = 'Moderate' WHERE movie_id = 'tt6848602'
UPDATE Master_Table SET MPAA_rating = 'Rated R for some disturbing violent images', nu

UPDATE Master_Table SET MPAA_rating = 'Rated PG for thematic elements, some violence and language', nudity = 'None', violence = 'None', profanity = 'None', alcohol = 'None', frightening = 'None' WHERE movie_id = 'tt5319866'
UPDATE Master_Table SET MPAA_rating = 'Rated R for language and some drug material', nudity = 'Mild', violence = 'Mild', profanity = 'Severe', alcohol = 'Moderate', frightening = 'Moderate' WHERE movie_id = 'tt8267604'
UPDATE Master_Table SET MPAA_rating = 'Rated R for graphic nudity, sexual content and language', nudity = 'Moderate', violence = 'Mild', profanity = 'Severe', alcohol = 'Mild', frightening = 'Mild' WHERE movie_id = 'tt5666304'
UPDATE Master_Table SET MPAA_rating = 'Rated R for disturbing violent images, language throughout, and brief sexuality/nudity', nudity = 'Moderate', violence = 'Moderate', profanity = 'Moderate', alcohol = 'Moderate', frightening = 'Moderate' WHERE movie_id = 'tt2368254'
UPDATE Master_Table SET MPAA_rating = 'Rated R for sexual 

UPDATE Master_Table SET MPAA_rating = 'None', nudity = 'None', violence = 'None', profanity = 'None', alcohol = 'None', frightening = 'None' WHERE movie_id = 'tt6495388'
UPDATE Master_Table SET MPAA_rating = 'Rated R for language throughout, crude sexual references, some drug use and a brief nude image', nudity = 'Moderate', violence = 'None', profanity = 'Severe', alcohol = 'None', frightening = 'Mild' WHERE movie_id = 'tt6288250'
UPDATE Master_Table SET MPAA_rating = 'Rated PG - 13 for some violent content and  thematic material', nudity = 'None', violence = 'None', profanity = 'None', alcohol = 'None', frightening = 'None' WHERE movie_id = 'tt2482856'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for some language', nudity = 'None', violence = 'None', profanity = 'Moderate', alcohol = 'Mild', frightening = 'Mild' WHERE movie_id = 'tt5175450'
UPDATE Master_Table SET MPAA_rating = 'Rated PG-13 for some language and brief violence', nudity = 'None', violence = 'None', profanity = 

UPDATE Master_Table SET MPAA_rating = 'None', nudity = 'None', violence = 'Mild', profanity = 'Moderate', alcohol = 'Moderate', frightening = 'Mild' WHERE movie_id = 'tt6705860'


('None', 'None', 'Mild', 'Moderate', 'Moderate', 'Mild')

In [228]:
def parse_user_ratings(movie_id):
    url = "https://www.imdb.com/title/" + movie_id + "/ratings"
    soup = prettify_page(url)
    
    table = soup.find('table')
    
    rows = table.find_all('tr')
    
    vote_array = []
    for row in rows:
        votes = row.find('div', class_='leftAligned').string
        if votes != 'Votes':
            vote_array.append(votes)
            
    reverse_votes = vote_array[::-1]
    return reverse_votes
    

In [229]:
parse_user_ratings('tt1825683')

['18,524',
 '5,648',
 '7,316',
 '12,274',
 '27,690',
 '64,113',
 '130,938',
 '134,400',
 '70,356',
 '83,169']

In [241]:
def parse_all_uratings(movie_dict):
    counter = 0

    for movie_id in movie_dict.keys():
        parsed_list = parse_user_ratings(movie_id)
        
        print(parsed_list[0])
        
        
        data_user_rating = {
        'movie_id': movie_id,
         'one': parsed_list[0],
        'two': parsed_list[1],
        'three': parsed_list[2],
        'four': parsed_list[3],
        'five': parsed_list[4],
        'six': parsed_list[5],
        'seven': parsed_list[6],
        'eight': parsed_list[7],
        'nine': parsed_list[8],
        'ten': parsed_list[9],
         }
        
        print(data_user_rating)


        inserting = """INSERT INTO User_Ratings  
         (movie_id, one, two, three, four, five, six, seven, eight, nine, ten) 
         VALUES (%(movie_id)s, %(one)s, %(two)s, %(three)s, %(four)s, %(five)s, %(six)s, %(seven)s, %(eight)s, %(nine)s, %(ten)s);"""

        cursor.execute(inserting, data_user_rating)
        cnx.commit()
        print(inserting)
            
            

        if counter%20 == 0:
            time.sleep(10)
        else:
            time.sleep(1)
        counter += 1
        
        break

In [243]:
parse_all_uratings({"tt1825683":True})

18,524
{'movie_id': 'tt1825683', 'one': '18,524', 'two': '5,648', 'three': '7,316', 'four': '12,274', 'five': '27,690', 'six': '64,113', 'seven': '130,938', 'eight': '134,400', 'nine': '70,356', 'ten': '83,169'}
INSERT INTO User_Ratings  
         (movie_id, one, two, three, four, five, six, seven, eight, nine, ten) 
         VALUES (%(movie_id)s, %(one)s, %(two)s, %(three)s, %(four)s, %(five)s, %(six)s, %(seven)s, %(eight)s, %(nine)s, %(ten)s);
